# Combine Routes

Merges the CSV route data with the JSON route data.

In [1]:
import pandas as pd
import re
import os
import json

## Read in the CSV data

In [2]:
scraped = pd.read_csv("scraped.csv")

## Read in the JSON data

Define a function to create a pandas dataframe from the `"routes"` entry in the JSON file.

In [4]:
def json_to_df(json_file):
    json_dict = json.load(json_file)
    just_routes = json_dict["routes"]
    return pd.DataFrame(just_routes)

Go through every JSON file in the folder, unioning their route data into one dataframe.

In [5]:
directory = "json-routes"
requested = []
for file in os.scandir(directory):
    with open(file.path, "r") as json_file:
        if len(requested):
            requested = pd.concat([requested, json_to_df(json_file)])
        else:
            requested = json_to_df(json_file)

## Merge the two datasets

A lot of the data is overlapping, but there are a few features the API-pulled data has that the scraped data doesn't and vice-versa.

In [6]:
print(" scraped:\n\n", scraped.columns, "\n\n", "requested:\n\n", requested.columns)

 scraped:

 Index(['Unnamed: 0', 'Route', 'Location', 'URL', 'Avg Stars', 'Your Stars',
       'Route Type', 'Rating', 'Pitches', 'Length', 'Area Latitude',
       'Area Longitude'],
      dtype='object') 

 requested:

 Index(['id', 'name', 'type', 'rating', 'stars', 'starVotes', 'pitches',
       'location', 'url', 'imgSqSmall', 'imgSmall', 'imgSmallMed', 'imgMedium',
       'longitude', 'latitude'],
      dtype='object')


Specifically, the **scraped** data has the vertical *length* of the route (in feet), the *rating* I've personally given routes I've climbed, and the "*Area*" *lat/lon* instead of the *route lat/lon*. 

From a quick inspection, the lat/lon values are the same, if truncated to fewer decimal places in the scraped data.

The **requested** data, on the other hand, has a *starVotes* column (a nice proxy for traffic) and the *image* links.

Before merging, I want to check for discrepancies between the two datasets. I'll start by adding a *route_id* column to the scraped dataframe.

In [7]:
get_id = lambda entry: int(re.search("\d{9}", entry).group())
scraped["id"] = scraped["URL"].apply(get_id)

The "indices" for both dataframes are not unique, so I'll use the route ID as the index instead.

In [8]:
scraped.index = scraped["id"]
requested.index = requested["id"]

Now we can order them by this new index and start comparing various columns' values.

In [44]:
scraped_sorted = scraped.sort_index()
requested_sorted = requested.sort_index()
all(scraped_sorted.index == requested_sorted.index)

True

The above assures that all route_ids match up.

Here, I'm going to define a little function that checks whether or not all values of features that are shared by both datasets match one another.

In [116]:
def check_same(scraped_col, requested_col):
    same = all(scraped_sorted[scraped_col] == requested_sorted[requested_col])
    if same:
        msg = "are the same"
    else:
        msg = "don't match"
    return "{} and {} {}\n".format(scraped_col, requested_col, msg)

In [117]:
scraped_cols = ["Route", "Location", "URL", "Avg Stars", "Route Type", "Rating", "Pitches", "Area Latitude", "Area Longitude"]
requested_cols = ["name", "location", "url", "stars", "type", "rating", "pitches", "latitude", "longitude"]

for i in range(len(scraped_cols)):
    print(check_same(scraped_cols[i], requested_cols[i]))

Route and name are the same

Location and location don't match

URL and url are the same

Avg Stars and stars don't match

Route Type and type are the same

Rating and rating are the same

Pitches and pitches don't match

Area Latitude and latitude don't match

Area Longitude and longitude don't match



For the ones that match, we'll just drop the column with the less-descriptive name. 
As for the non-matching items, let's see what's different about each of these.

**Location**

In [85]:
print(scraped_sorted["Location"].head(1), requested_sorted["location"].head(1))

id
105888384    The Dihedrals > Foster Falls > Tennessee
Name: Location, dtype: object id
105888384    [Tennessee, Foster Falls, The Dihedrals]
Name: location, dtype: object


Because it's really just the formatting, I'll discard the first and keep the second, more convenient version.

**Stars**

In [86]:
print(scraped_sorted["Avg Stars"].head(3), requested_sorted["stars"].head(3))

id
105888384    2.8
105888407    2.6
105892528    2.2
Name: Avg Stars, dtype: float64 id
105888384    3.8
105888407    3.6
105892528    3.2
Name: stars, dtype: float64


These all differ by one. The API usage page explains that the former rating corresponds to the star rating a user might select on the app (ranges 1-4, inclusive) while the latter rating is adjusted to range 1-5, inclusive.

It appears, however, that NA values have been imputed as -1 in the scraped dataset and as 0 in the requested dataset:

In [105]:
requested_sorted[requested_sorted["stars"] == 0] # Only where starVotes also equals zero

,id,name,type,rating,stars,starVotes,pitches,location,url,imgSqSmall,imgSmall,imgSmallMed,imgMedium,longitude,latitude
id,,,,,,,,,,,,,,,
111607012,111607012,The Peanut,Sport,5.12c,0.0,0,1,"[Alabama, Sand Rock, The Islands]",https://www.mountainproject.com/route/11160701...,,,,,-85.8193,34.1792
111607023,111607023,Little Jerk,Trad,5.11c/d,0.0,0,1,"[Alabama, Sand Rock, The Islands]",https://www.mountainproject.com/route/11160702...,,,,,-85.8193,34.1792
112293128,112293128,Feet don't fail me now,Sport,5.11a/b,0.0,0,1,"[Tennessee, Foster Falls, Music City/The 5.9 A...",https://www.mountainproject.com/route/11229312...,,,,,-85.7010,35.1869
113973514,113973514,Katana,Sport,5.12a,0.0,0,1,"[Tennessee, Foster Falls, Music City/The 5.9 A...",https://www.mountainproject.com/route/11397351...,,,,,-85.7010,35.1869
117964031,117964031,Lee Trevino,Sport,5.11c,0.0,0,1,"[Alabama, Little River Canyon, Lizard Wall]",https://www.mountainproject.com/route/11796403...,,,,,-85.6702,34.3588


Hate that. I'm going to just keep the *Avg Stars* version in the range from 1-4.

**Pitches**

If a route has no pitch information in the scraped dataset, its default is 1. In the requested dataset, its default is an empty string "". The first is going to be far more reasonable as multi-pitch routes will probably be more established and well-documented on the site.

**Longitude/Latitude**

In [131]:
print(scraped_sorted["Area Longitude"].head(), requested_sorted["longitude"].head())

id
105888384   -85.68004
105888407   -85.68366
105892528   -85.68247
105892533   -85.67854
105892538   -85.68420
Name: Area Longitude, dtype: float64 id
105888384   -85.6800
105888407   -85.6837
105892528   -85.6825
105892533   -85.6785
105892538   -85.6842
Name: longitude, dtype: float64


It looks like the values have just been rounded in the API dataset. So I'll keep the extra decimal place and stick with the scraped version.

In summary, I'll merge the datasets on their indices and drop the following columns:
Route, Location, URL, stars, type, Rating, pitches, latitude, longitude.

I'll also drop the obselete index titled "unnamed: 0" and the redundant "id" columns.

In [154]:
merged = pd.concat([scraped_sorted, requested_sorted], axis=1)
for col in ["Route", "Location", "URL", "stars", "type", "Rating", "pitches", "latitude", "longitude", "id", "Unnamed: 0" ]:
    merged.drop(col, axis=1, inplace=True)
merged.columns

Index(['Avg Stars', 'Your Stars', 'Route Type', 'Pitches', 'Length',
       'Area Latitude', 'Area Longitude', 'name', 'rating', 'starVotes',
       'location', 'url', 'imgSqSmall', 'imgSmall', 'imgSmallMed',
       'imgMedium'],
      dtype='object')

## Write to file

In [155]:
with open("merged.csv", "w") as outfile:
    merged.to_csv(outfile)

Next, I'll run some exploratory analyses on this combined dataset.